# Si446x Device Direct Access TagNet Software Image Load

In [ ]:
from __future__ import print_function
from builtins import *                  # python3 types
from time import sleep
from datetime import datetime
import struct as pystruct
from binascii import hexlify
import os.path

In [ ]:
!pwd
%autosave 0
import sys
sys.path.append("../si446x/si446x")
%run '../si446x/si446x/notebooks/si446x_Device_Layer.ipynb'

In [ ]:
import sys
sys.path.append("../tagnet/tagnet")
from tagmessages import TagMessage, TagPoll, TagGet, TagPut, TagDelete, TagHead
from tagnames import TagName
from tagtlv import TagTlv, TagTlvList, tlv_types

In [ ]:
import datetime
print('Test Start Time: {}'.format(datetime.datetime.now()))
print('Si446x Radio Device Driver Version: {}'.format(si446x_device_version()))

##  Start up Radio

In [ ]:
radio = si446x_device_start_radio()

In [ ]:
si446x_device_show_config(radio.dump_radio())

## Check for Command Error

In [ ]:
status = radio.get_chip_status()
if (status.chip_pend.CMD_ERROR):
    print(status)

##  Configure Radio

In [ ]:
config = si446x_device_config_radio(radio)

si446x_device_show_config(radio.dump_radio())
total = 0
print('\n=== const config strings:')
for s in config:
    print((hexlify(s)))
    total += len(s) - 4
print('\n total: {}'.format(total))

## Transfer Software Image using TagNet

### Image Info description
Image Description Information stored in the Image File

In [ ]:
#filename    = '/tmp/test.bin'
filename    = 'main.bin'

In [ ]:
#  IMAGE_INFO provides information about a Tag software image. This data is
#  embedded in the image itself. The IMAGE_META_OFFSET is the offset into
#  the image where image_info lives in the image.  It directly follows the
#  exception vectors which are 0x140 bytes long.
# 
#  This struct will have to change, If MSP432 vector table length changes.
# 
IMAGE_INFO_SIG = 0x33275401
IMAGE_META_OFFSET = 0x140
IMAGE_INFO_DEFAULT = [IMAGE_INFO_SIG, 0x20000, (0x140*2)+0x1c, 0, 0, 257, 2, 3, ' '  * 40, ' '  * 40, ' '  * 30, 0xde, 2]
#
# Struct created for accessing image info (little indian)
# sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major, main_tree, aux_tree, build_time, im_rev, im_model = image_info
#
IM_FIELDS = '<LLLLLHBB44s44s30sBB'
image_info_struct = pystruct.Struct(IM_FIELDS)
IMAGE_MIN_SIZE  =  (IMAGE_META_OFFSET + image_info_struct.size)

In [ ]:
# write out simple default binary input file for testing purposes
#
if not os.path.isfile(filename):
    with open(filename,'wb') as outfile:
        buf = bytearray(IMAGE_META_OFFSET)
        for x in range(1,IMAGE_META_OFFSET): buf[x] = x & 0x7f
        outfile.write(buf)
        outfile.write(bytearray(image_info_struct.pack(*IMAGE_INFO_DEFAULT)))
        for i in range(64):
            for x in range(256): buf[x] = x & 0x7f
            outfile.write(buf)

In [ ]:
try:
    infile = open(filename, 'rb')
    infile.seek(0, 2) # seek to the end
    file_size = infile.tell()
    if file_size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    infile.seek(0, 0)    # seek to the beginnnig
    # get image info from input file and sanity check
    infile.seek(IMAGE_META_OFFSET) # seek to location of image info
    image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
    print("file information")
    sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major,\
        main_tree, aux_tree, build_time, im_rev, im_model = image_info
    pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, vect_chk: 0x{:x}, image_chk: 0x{:x}"
    print(pstr.format(sig, image_start, imagelength, vector_chk, image_chk))
    pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
    print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
    if sig != IMAGE_INFO_SIG: raise RadioLoadException("image metadata is invalid")
    infile.seek(0)    # seek to the beginning
    
finally:
    infile.close()

### utility routines for handling image load

In [ ]:
# debug flags
PRINT_MAX = 80

In [ ]:
# default paramters
MAX_WAIT            = 10
MAX_RECV            = 255
MAX_PAYLOAD         = 254
MAX_RETRIES         = 10
RADIO_POWER         = 100
SHORT_DELAY         = 0

In [ ]:
# build Image PUT Request
def im_send_request(fd, write_max, vers, eof=False):
    # base image name
    req_name = TagName ('/tag/sd') \
                + TagTlv(tlv_types.NODE_ID, -1) \
                + TagTlv(0) \
                + TagTlv('img') \
                + TagTlv(tlv_types.VERSION, vers)
    # optionally add offset to name
    if (fd.tell() > 0):
        req_name = req_name + TagTlv(tlv_types.OFFSET, fd.tell())
    # build the PUT mesage object
    req_obj = TagPut(req_name)
    # optionally add payload
    if eof:
        # send end of file indication
        pload = TagTlvList([TagTlv(tlv_types.EOF)])
    elif (fd.tell() < write_max):
        # determine payload size to send and read it
        chunk_size = MAX_PAYLOAD - req_obj.pkt_len()
        if ((file_size - fd.tell()) < chunk_size):
            chunk_size = file_size - fd.tell()
        pload = bytearray(fd.read(chunk_size))
    # else
        # just send without payload
    
    # print out details of request
#    print(req_obj.header)
#    print(req_obj.name)
    if (pload is not None):
        req_obj.payload = pload
    req_msg = req_obj.build()
#    print("   ", hexlify(req_msg[:PRINT_MAX]),"...", hexlify(req_msg[-PRINT_MAX:]))
    
#    print("REQUEST MSG,  msg len: {},  payload len: {}".format(len(req_msg), len(pload)))
    # send request msg
    si446x_device_send_msg(radio, req_msg, RADIO_POWER);
    del req_name
    return req_obj, req_msg, len(pload)

In [ ]:
req_name = TagName ('/tag/sd') \
                + TagTlv(tlv_types.NODE_ID, -1) \
                + TagTlv(0) \
                + TagTlv('img') \
                + TagTlv(tlv_types.VERSION, (0,0,0))
req_obj = TagPut(req_name)
req_obj.payload = TagTlvList([TagTlv(tlv_types.EOF)])
req_msg = req_obj.build()
#print(hexlify(req_msg))

In [ ]:
def im_get_response(fd):
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, MAX_WAIT)
    if (not rsp_buf):
        return None, None, -1, 0
    rsp_obj = TagMessage(rsp_buf)
    # get offset
    offset = 0
    if (rsp_obj.payload) and (rsp_obj.payload[0].tlv_type() == tlv_types.OFFSET):
        offset = rsp_obj.payload[0].value()
    # print out details of response
#    print()
#    print("RESPONSE MSG, error: {}".format(rsp_obj.header.options.param.error_code))
#    print(rsp_obj.header)
#    print(rsp_obj.name)
#    if (rsp_obj.payload):
#        print(rsp_obj.payload)
#        print("   offset: {}, msg len: {},  payload len: {}, rssi: {}".format(offset, len(rsp_buf), 0, rssi))
#        print("   ", hexlify(rsp_buf[:PRINT_MAX]),"...", hexlify(rsp_buf[-PRINT_MAX:]))
    return rsp_obj, rsp_buf, offset, rssi;

In [ ]:
from __future__ import print_function
from datetime import datetime

### main funtion for transfer of image to tag

In [ ]:
radio.trace._enable()
# set rssi control (enable latch, average4, latch on sync)
prp         = bytearray('0x22')
radio.set_property('MODEM', 0x4c, prp) 
# tx/rx threshold
#prp = bytearray('\x20\x20')
#radio.set_property('PKT', 0x0b, prp)
start = datetime.now()
print(start)
class RadioLoadException(Exception):
    pass
try:
    # open input file and determine its length
    infile = open(filename, 'rb')
    infile.seek(0, 2) # seek to the end
    file_size = infile.tell()
    if file_size < IMAGE_MIN_SIZE: raise RadioLoadException("input file too short")
    infile.seek(0, 0)    # seek to the beginnnig
    # get image info from input file and sanity check
    infile.seek(IMAGE_META_OFFSET) # seek to location of image info
    image_info = image_info_struct.unpack(infile.read(image_info_struct.size))
    print("file information")
    sig, image_start, imagelength, vector_chk, image_chk, im_build, im_minor, im_major,\
        main_tree, aux_tree, build_time, im_rev, im_model = image_info
    pstr = "  signature: 0x{:x}, start: 0x{:x}, length: 0x{:x}, vect_chk: 0x{:x}, image_chk: 0x{:x}"
    print(pstr.format(sig, image_start, imagelength, vector_chk, image_chk))
    pstr = "  version: ({}.{}.{}(0x{:x})), rev: {}, model: {}"
    print(pstr.format(im_major, im_minor, im_build, im_build, im_rev, im_model))
    if sig != IMAGE_INFO_SIG: raise RadioLoadException("image metadata is invalid")
    infile.seek(0)    # seek to the beginnnig
    # loop to transfer image data to tag
    retries         = 0
    total_retries   = 0
    packets_sent    = 0
    rssi            = 0
    while (file_size - infile.tell() > 0):
        if ((packets_sent % 10) == 0):
            print("\r{0:.2f}%".format((float(infile.tell())/float(file_size))*100), end='')
#        print(">>>> file size: {}, offset: {}, retries: {}, rssi: {}\r".format(file_size, infile.tell(), retries, rssi, end=''))
        __, __, plen = im_send_request(infile, file_size, (im_build, im_minor, im_major))
        packets_sent += 1
#        print("----")
        rsp_obj, rsp_msg, offset, rssi = im_get_response(infile)
        # check that offset is expected
        if offset != infile.tell():
            if (rsp_obj is None): p = "TIMEOUT"
            else:                 p = "BAD OFFSET"
            print("\n{}  {}! offset: response {}, expected {}".format(datetime.now(), p, offset, infile.tell()))
            if (offset >= 0):
                infile.seek(offset, 0)
            else:
                infile.seek(-plen, 1)
                offset -= plen
                
            retries = retries + 1
            total_retries = total_retries + 1
        else:
            retries = 0
        if (retries > MAX_RETRIES):
            raise RadioLoadException("too many retries")
        sleep(SHORT_DELAY)
    # send end of file to complete the image load operation
    im_send_request(infile, file_size, (im_build, im_minor, im_major), True)
    rsp_obj, rsp_msg, offset, rssi = im_get_response(infile)
    print("\ntotals bytes: {}, packets: {} retries: {}".format(offset, packets_sent, total_retries))
except RadioLoadException:
    print('terminating')
finally:
    infile.close()
print('\ndone, elapsed time', datetime.now() - start)

In [ ]:
print(hexlify(rsp_buf))
print(offset)
print(rsp_obj.header)
print(rsp_obj.header.options.param.error_code)
print(rsp_obj.name)
print(rsp_obj.payload)

## Get Directory

In [ ]:
#"tag"  "sd"    <node_id>   "img"
image_manager_name = TagName ('/tag/sd') \
                + TagTlv(tlv_types.NODE_ID, -1) \
                + TagTlv(0) \
                + TagTlv('img')
dir_info = TagGet(image_manager_name)
#print(dir_info.name)
dir_msg = dir_info.build()
#print(len(dir_msg),hexlify(dir_msg))
si446x_device_send_msg(radio, dir_msg, RADIO_POWER);
rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, MAX_WAIT)
if (rsp_buf):
    print(len(rsp_buf),hexlify(rsp_buf))
    rsp_obj = TagMessage(rsp_buf)
    for x in range(0, 8, 2):
        print("state: {}, {}".format(rsp_obj.payload[x+1].value(), rsp_obj.payload[x]))
else:
    print('timeout')

## Get Versions

In [ ]:
#"tag"	"sys"	<node_id>	"which"
def get_version(which):
    sys_name = TagName ('/tag/sys') \
                        + TagTlv(tlv_types.NODE_ID, -1) \
                        + TagTlv(which)
    sys_obj = TagGet(sys_name)
#    print(sys_obj.name)
    get_msg = sys_obj.build()
    si446x_device_send_msg(radio, get_msg, RADIO_POWER);
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_buf):
#        print(hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
        print("{} {:^10} state: {}, {}".format(rsp_obj.header.options.param.error_code, \
                                                         which, rsp_obj.payload[1].value(),       \
                                                         rsp_obj.payload[0]))

In [ ]:
get_version('active')
get_version('backup')
get_version('golden')
get_version('nib')
get_version('running')

## Def Set Active/Backup Version

In [ ]:
#"tag"	"sys"	<node_id>	"which"
def set_version(which, version):
    set_name = TagName ('/tag/sys') \
                        + TagTlv(tlv_types.NODE_ID, -1) \
                        + TagTlv(which) \
                        + TagTlv(tlv_types.VERSION, version)
    set_obj = TagPut(set_name)
#    print(set_obj.name)
    set_msg = set_obj.build()
    si446x_device_send_msg(radio, set_msg, RADIO_POWER);
    rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
    if(rsp_buf):
#        print(hexlify(rsp_buf))
        rsp_obj = TagMessage(rsp_buf)
#        print(rsp_obj.header)
        if (rsp_obj.payload):
            print("{}: state: {}, {}".format(rsp_obj.header.options.param.error_code, rsp_obj.payload[1].value(), rsp_obj.payload[0]))
        else:
            print("{}".format(rsp_obj.header.options.param.error_code))

## Set Active Version

In [ ]:
set_version('active', (118, 1, 0))

## Set Backup Version

In [ ]:
set_version('backup', (112, 1, 0))

## Reboot Running Version

In [ ]:
set_version('running', (116, 1, 0))

## Reboot NIB

In [ ]:
set_version('nib', (115, 1, 0))

## Reboot Golden

In [ ]:
set_version('golden', (117, 1, 0))

## Delete Version

In [ ]:
#"tag"  "sd"    <node_id>   "img"   <version>
delete_image_name = TagName ('/tag/sd') \
                + TagTlv(tlv_types.NODE_ID, -1) \
                + TagTlv(0) \
                + TagTlv('img') \
                + TagTlv(tlv_types.VERSION, (4, 1, 0))
delete_info = TagDelete(delete_image_name)
#print(delete_info.header)
print(delete_info.name)
delete_msg = delete_info.build()
si446x_device_send_msg(radio, delete_msg, RADIO_POWER);
rsp_buf, rssi, status = si446x_device_receive_msg(radio, MAX_RECV, 5)
if(rsp_buf):
#    print(hexlify(rsp_buf))
    rsp_obj = TagMessage(rsp_buf)
#    print(rsp_obj.header.options.param.error_code)
#    print(rsp_obj.payload)
    if (rsp_obj.payload):
        print("{}: state: {}, {}".format(rsp_obj.header.options.param.error_code, rsp_obj.payload[1].value(), rsp_obj.payload[0]))
    else:
        print("{}".format(rsp_obj.header.options.param.error_code))

## Reboot Tag

In [ ]:
set_version('running', (44, 16, 0))

## Get Chip Status

In [ ]:
print(radio.get_chip_status())

## Interactive Group Properties

In [ ]:
from ipywidgets import interact
interact(si446x_device_group_fetch_and_decode, group=radio_config_group_ids.encoding)

## Interactive  Command Status Responses

In [ ]:
interact(si446x_device_command_fetch_and_decode, cmd=radio_status_cmd_ids.encoding)

In [ ]:
from datetime import datetime
datetime.now()